In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D
from keras import optimizers
from keras.optimizers import Adadelta,Adam


#from keras.preprocessing import sequence
from keras.losses import categorical_crossentropy


from google.colab import drive
drive.mount('/content/gdrive')

Using TensorFlow backend.


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
data =pd.read_csv('gdrive/My Drive/fer2013.csv')
data.head()   
pixels = data['pixels'].tolist()
final_pixels_list=[]
for i in range(len(pixels)):
  pixels1=pixels[i].split()
  results = [int(j) for j in pixels1]
  array = np.array(pixels1)
  array=array.reshape(48,48,1)
  array = np.array(array, dtype=np.uint8)
  final_pixels_list.append(array)     
training_count=0
testing_count=0
validation_count=0
for i in range(len(pixels)):
  if data['Usage'][i]=="Training":
    training_count=training_count+1
  elif data['Usage'][i]=='PublicTest':
    validation_count=validation_count+1
  else:
    testing_count=testing_count+1  

print(training_count)
print(validation_count)
print(testing_count)
print(data.tail())
test_output=data['emotion'].tolist()
for i in range(len(test_output)):
     test_output[i]=int(test_output[i])
encoded = to_categorical(test_output)
test_output=encoded

X_train=final_pixels_list[0:training_count]
X_val = final_pixels_list[training_count: training_count+validation_count]
X_test=final_pixels_list[training_count+validation_count:len(final_pixels_list)]

y_train=test_output[0:training_count]
y_val = test_output[training_count:training_count+validation_count]
y_test=test_output[training_count+validation_count:len(final_pixels_list)]

print(len(X_train))
print(len(X_val))
print(len(X_test))
print((y_train.shape))
print(len(y_val))
print(len(y_test))

28709
3589
3589
       emotion                                             pixels        Usage
35882        6  50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...  PrivateTest
35883        3  178 174 172 173 181 188 191 194 196 199 200 20...  PrivateTest
35884        0  17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...  PrivateTest
35885        3  30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...  PrivateTest
35886        2  19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...  PrivateTest
28709
3589
3589
(28709, 7)
3589
3589


In [3]:
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)

y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

print("X: train, val, test shapes: ",X_train.shape, X_val.shape, X_test.shape)
print("Y: train, val, test shapes: ",y_train.shape, y_val.shape, y_test.shape)

X: train, val, test shapes:  (28709, 48, 48, 1) (3589, 48, 48, 1) (3589, 48, 48, 1)
Y: train, val, test shapes:  (28709, 7) (3589, 7) (3589, 7)


In [4]:
from keras.models import model_from_json

batch_size=300

h5_filename = 'gdrive/Team Drives/ML project/Manasa/model.h5'
json_filepath = 'gdrive/Team Drives/ML project/Manasa/model.json'
json_file = open(json_filepath,'r')

loaded_model_json = json_file.read()

json_file.close()

saved_model = model_from_json(loaded_model_json)
saved_model.load_weights(h5_filename)

saved_model.summary()

ada = Adadelta(lr=0.05, rho=0.95, epsilon=1e-08)
saved_model.compile(loss='categorical_crossentropy',
              optimizer=ada,
              metrics=['accuracy'])



test_scores = saved_model.evaluate(X_test, y_test, batch_size=batch_size)

print("Test Loss: " + str(test_scores[0]))
print("Test Accuracy: " + str(test_scores[1]))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 44, 44, 64)        1664      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 22, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 128)       73856     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 128)       147584    
__________

In [7]:
from keras.models import Model

layer_name = 'dropout_2'

intermediate_layer_model = Model(inputs=saved_model.input, outputs=saved_model.get_layer(layer_name).output)

train_features = intermediate_layer_model.predict(X_train)
train_labels = np.argmax(y_train,axis=1)

print(train_features.shape)
print(train_labels.shape)

val_features = intermediate_layer_model.predict(X_val)
val_labels = np.argmax(y_val,axis=1)

print(val_features.shape)
print(val_labels.shape)

test_features = intermediate_layer_model.predict(X_test)
test_labels = np.argmax(y_test,axis=1)

print(test_features.shape)
print(test_labels.shape)


(28709, 1024)
(28709,)
(3589, 1024)
(3589,)
(3589, 1024)
(3589,)


In [8]:
from sklearn.svm import SVC
  
clf = SVC(C=1.0, kernel="rbf")
clf.fit(train_features, train_labels)


pred_test_label = clf.predict(test_features)
num = len(pred_test_label)
accuracy = len([1 for i in range(num) if test_labels[i]==pred_test_label[i]])/float(num)

print("svm on the top of cnn -  %.2f%% accuracy:" % (accuracy * 100))


svm on the top of cnn -  60.32% accuracy:
